In [1]:
import pandas as pd
from fuzzywuzzy import process


import os
os.chdir("../")

EXCEL = "./data/Jury/Bewertung/Master Sheet/Master Sheet.xlsx"
JSON = "./data/texts.json"
COMBINED = "./data/combined.json"

In [2]:
df2 = pd.read_excel(EXCEL, header=1)
df2['Startup'] = df2['Startup'].str.split(" -").str.get(0)

In [3]:
df1 = pd.read_json(JSON)
df1.head()

,Filename,Description,Difference,1. Your company/vision/mission in a tweet ! *,2. Which sector/subsector of Fintech do you operate in?,3. SStage of company *,4. Which problem does your company solve? *,5. What sparked the founder(s) to set up the company? *,"6. Please describe the business model of your company, including the main sources of revenue. *",7. What is your USP versus traditional competitors or those in the digital world? *,...,12. Please provide a brief competition landscape for your core markets. *,13. Did you pivot in the past and why?,14. Which business area in the last year consumed most of your management attention? *,15. What protects your business from new market entrants or copycats? *,"16. After your next financing round, will the founders jointly hold above or below 50% of the",17. Anything else you would want to add or let us know?,18. How do you integrate ESG standards in your business model? *,"19. When scaling the company, how do you ensure that the corporate culture does not suffer and",20. Please upload your startup logo. *,"21. Did your firm apply to FTGA before? If yes, which year? *"
0,21financeAG.pdf,21finance provides financial institutions with...,We strive to enhance the accessibility and ef...,E\n21fiSnance is a whitelabel-service provider...,E\nS\nBlocSkchain\nP\nCrowdfunding/-investing\...,SE\nSeeSd Stage\nP\nOR,"NE\nAccSSessibility to financial instruments, ...",E\nFouSnder & CEO: Max J. Heinzle holds an M.S...,E\n21fiSnance is a full-service provider for t...,E\nOurS main differentiation stems from our co...,...,SE\nAs aES full-service provider of digital as...,"E\nIn tShe history of 21finance, we pivoted tw...","E\nTheS Application, setup and development of ...",E\nIn cSomparison to other providers that have...,E\nshaSres? *\nBeloPw 50% of the shares\nO\nNR,"SE\nOn ESthe technology side, we enable intero...",E\nEngSagement in responsible investment pract...,E\nthaSt the shared values and goals remain in...,E\nYoSur logo will be uploaded on the award we...,E\n- S\nP\nO\nN\nS\nE\nS
1,21Shares.pdf,Our objective is to make cryptocurrencies acce...,21Shares makes investing in crypto accessible...,E\nLeaSding the way to an easier crypto experi...,NE\nBlocSSkchain\nEP\nSOR,NE\nSerSSies A\nEP\nSOR,NE\n21SSShares looks to provide access to cryp...,"E\nHanSy and Ophelia, the company’s co-founder...",E\n21SShares is the world's largest issuer of ...,"E\n21SShares launched HODL, the first ETP that...",...,NE\nWhSSile there are other crypto ETP issuers...,SE\n- ES\nSP\nOR,NE\nWe SSare focused on launching new products...,SE\nWe ESare the world’s largest provider of c...,E\nshaSres? *\nNotP available to disclose.\nO\...,E\n- S\nP\nOR,NE\n21SSShares believes that the crypto market...,E\nthaSt the shared values and goals remain in...,SE\nYoESur logo will be uploaded on the award ...,E\nNoS\nP\nO\nN\nS\nE\nS
2,21st RealEstate.pdf,,-,E\nWirS digitalisieren den Bewertungs- und Tra...,E\nS\nProSptech\nP\nOR,NE\nPubSSlicly listed\nEP\nSOR,NE\nVielSSe Unternehmen beschäftigen sich mit ...,"E\nDieS ursprünglichen Gründer, die heute auch...",E\nIn RSELAS können einzelne Adressen oder Por...,E\nA) FSlächendeckende Preisinformationen auch...,...,NE\nEs gSSibt einige Mitbewerber für Standorta...,E\nJa. BSei Unternehmensgründung lag der Fokus...,E\nDieS Bereitstellung ESG-konformer Daten war...,E\nWirS sind ein etabliertes PropTech-Unterneh...,E\nshaSres? *\nneinP. Den größten Anteil hält ...,"E\nS\nWirS sind das erste PropTech, welches Ch...","E\nWirS haben aktuell über 70 Lageindikatoren,...",E\nthaSt the shared values and goals remain in...,E\nYoSur logo will be uploaded on the award we...,E\nnoS\nP\nO\nN\nS\nE\nS
3,3Pace.pdf,"With our IIINITY® Algorithms, we offer AI base...",We are the world's first provider of AI-assis...,E\nWe Soffer the worldwide smartest withdrawal...,NE\nArtiSSficial Intelligence\nEP\nDigital Wea...,E\nBooStstrapped\nP\nOR,NE\nLowSS pensions and lack of solutions for t...,NE\nOwSSn expe

In [4]:
# Function to find the best match using fuzzy matching
def fuzzy_match(target, choices, threshold=80):
    match, score = process.extractOne(target, choices)
    if score >= threshold:
        return match
    return None

# Extract relevant identifiers from `Filename` (without file extension)
df1['SearchKey'] = df1['Filename'].str.replace('.pdf', '', regex=False)

# Apply fuzzy matching to df1['SearchKey'] against all df2['Startup']
df1['BestMatch'] = df1['SearchKey'].apply(
    lambda x: fuzzy_match(x, df2['Startup'].tolist())
)

# Merge DataFrames based on the fuzzy match result
df = df1.merge(df2, left_on='BestMatch', right_on='Startup', how='left')

In [5]:
df = df.dropna(subset=['Startup'])
df = df.dropna(subset=['Filename'])

df.to_json(COMBINED, orient='records')